### 1. Imports

In [227]:
import yfinance as yf  
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.losses import MSE
from keras.optimizers import Adam
import os

sns.set()

### 2. Loading in Dataset

In [319]:
import matplotlib.pyplot as plt
df = yf.download('TSLA','2010-06-29','2019-05-28',interval='1d')


[*********************100%***********************]  1 of 1 downloaded


In [320]:
df1 = df.copy()

In [321]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,19.00,25.00,17.54,23.89,23.89,18766300
2010-06-30,25.79,30.42,23.30,23.83,23.83,17187100
2010-07-01,25.00,25.92,20.27,21.96,21.96,8218800
2010-07-02,23.00,23.10,18.71,19.20,19.20,5139800
2010-07-06,20.00,20.00,15.83,16.11,16.11,6866900
